In this notebook, we are going to explore the eigenstate thermalization hypothesis (ETH) and thermalization in quenches from initial states.

Some useful references:
1. Srednicki, The approach to thermal equilibrium in quantized chaotic systems; https://arxiv.org/abs/cond-mat/9809360v2

2. Buegeling, Moessner, Haque, Finite-size scaling of eigenstate thermalization; https://arxiv.org/abs/1308.2862

3. D'Alessio, Kafri, Polkovnikov, Rigol, From Quantum Chaos and Eigenstate Thermalization to Statistical Mechanics and Thermodynamics; https://arxiv.org/abs/1509.06411 (longer review article)




# Useful Functions #

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sparse
from scipy import linalg
import time

import warnings
warnings.filterwarnings("ignore")

In [2]:
def gen_s0sxsysz(L):
    sx = sparse.csr_matrix([[0., 1.],[1., 0.]])
    sy = sparse.csr_matrix([[0.,-1j],[1j,0.]])
    sz = sparse.csr_matrix([[1., 0],[0, -1.]])
    s0_list =[]
    sx_list = []
    sy_list = []
    sz_list = []
    I = sparse.eye(2**L, format='csr', dtype='complex')
    for i_site in range(L):
        if i_site==0:
            X=sx
            Y=sy
            Z=sz
        else:
            X= sparse.csr_matrix(np.eye(2))
            Y= sparse.csr_matrix(np.eye(2))
            Z= sparse.csr_matrix(np.eye(2))

        for j_site in range(1,L):
            if j_site==i_site:
                X=sparse.kron(X,sx, 'csr')
                Y=sparse.kron(Y,sy, 'csr')
                Z=sparse.kron(Z,sz, 'csr')
            else:
                X=sparse.kron(X,np.eye(2),'csr')
                Y=sparse.kron(Y,np.eye(2),'csr')
                Z=sparse.kron(Z,np.eye(2),'csr')
        sx_list.append(X)
        sy_list.append(Y)
        sz_list.append(Z)
        s0_list.append(I)

    return s0_list, sx_list,sy_list,sz_list


def gen_op_total(op_list):
    L = len(op_list)
    tot = op_list[0]
    for i in range(1,L):
        tot = tot + op_list[i]
    return tot

def gen_op_prod(op_list):
    L= len(op_list)
    P = op_list[0]
    for i in range(1, L):
        P = P*op_list[i]
    return P

def gen_interaction_kdist(op_list, op_list2=[],k=1, bc='obc'):
    L= len(op_list)

    if op_list2 ==[]:
        op_list2=op_list
    H = sparse.csr_matrix(op_list[0].shape)
    Lmax = L if bc == 'pbc' else L-k
    for i in range(Lmax):
        H = H+ op_list[i]*op_list2[np.mod(i+k,L)]
    return H

def gen_state_bloch(thetaList, phiList):
    L=len(thetaList)
    psi = np.kron([np.cos(thetaList[0]/2.),np.exp(1j*phiList[0])*np.sin(thetaList[0]/2.)],
                  [np.cos(thetaList[1]/2.),np.exp(1j*phiList[1])*np.sin(thetaList[1]/2.)])
    for i in range(2,L):
        psi = np.kron(psi, [np.cos(thetaList[i]/2.),np.exp(1j*phiList[i])*np.sin(thetaList[i]/2.)])
    return psi


# Model #

As a model Hamiltonian, we'll work with the transverse field Ising model (TFIM) with an additional longitudinal field:

$H = J \sum_i \sigma_i^z \sigma_{i+1}^z + h_x\sum_i \sigma_i^x  + h_z \sum_i \sigma_i^z $

When $h_z=0$, this is the TFIM you looked at in the last HW. The TFIM can be re-written as a model of non-interacting Majorana fermions and is thus a `free' integrable model. The additional longitudinal field with strength $h_z$ makes the model non-integrable and breaks the Ising symmetry.

Choose your parameters as below:


In [3]:
J=1
hz = (np.sqrt(5)+1)/4.
hx = (np.sqrt(5)+5)/8.
BC = 'obc'

The choice of parameters above may seem strange. They are chosen from this paper: https://arxiv.org/abs/1306.4306 and these parameters have been found to make the model robustly non-integrable even at small sizes. While any non-zero $h_z$ is non-integrable, in practice one may need to go to very large sizes and/or very late times to see this.

# Question 1: Energy and Entropy (15 pts) #

a. Construct and diagonalize the Hamiltonian for the model above for L=12 (use the helper functions). Plot the normalized distribution of eigenenergies as a function of energy using plt.hist or np.histogram. This serves as a proxy for the density of states (DOS). [5pts]

b. What is the shape of the DOS? Next week, we will contrast this DOS with that of a dense random matrix, which has a semicircular DOS.  [1pt]

c. The bandwith of a spectrum is defined as the difference in energy between the highest and lowest energy states. How does the bandwidth scale with system size $L$? It is preferred for you to answer this part by general analtyic reasoning, but it is acceptable to answer by simulating different $Ls$. [1pt]

d. How does the standard deviation of the DOS scale with $L$? This represents the energy window where most of the eigenstates live, which scales differently from the bandwidth. It is preferred for you to answer this part by analtyic reasoning, but it is acceptable to answer by simulating different $Ls$. Your analytic answer can be given on general terms and does not need to specifically consider details of the model Hamiltonian.  [1pt]

Recall that the entropy is given by $S=\ln \Omega$, where $\Omega$ is the multiplicity of states consistent with a given set of thermodynamic parameters (such as energy, volume etc.), and we set $k_B = 1$. The density of states $N(E)$ is the number of eigenstates with energy $E$ per unit volume per unit energy and is a proxy for the entropy.

From the curve of entropy vs. energy, we can extract the temperature conjugate to each energy via $dS/dE = 1/T$. `Infinite temperature' corresponds to the energy density with the maximum entropy. Recall from your stat mech class that temperatures can be negative and positive (the slopes $dS/dE$ for your plot can be both positive and negative). A negative temperature $-T_0$ for $H$ simply corresponds to positive temperature $T_0$ for the Hamiltonian $-H$.

Indeed, we can find the equilibrium energy corresponding to a given temperature in the usual way using the Gibbs ensemble:
$\langle E\rangle_T =  \frac{1}{Z} \sum_\alpha e^{-E_\alpha/T}E_\alpha$, where the sum is over the complete basis of energy eigenstates $|E_\alpha\rangle$ and we have assumed there are no continuous symmetries other than energy.  

Finally, note that temperature is intensive and determined by the energy *density*, $E/V$. Recall that the uncertainty in the average energy at a given temperature in the canonical ensemble is $\Delta E \propto \sqrt{V}$. Thus, all states in a large window $\Delta E$ around $\langle E\rangle_T$ have the same temperature.  


e. What is the temperature corresponding to the ground state (i.e. the state at the bottom of the many-body spectrum?) [1pt]

f. What is the energy corresponding to infinite temperature? Where does this energy live in the many-body spectrum? [1pt]

g. What is the temperature corresponding to the highest excited state (i.e. the state at the top of the many-body spectrum?) [1pt]

h. Consider temperatures $T=0.1, 1, 10, 100, -100, -10, -1, -0.1$. Numerically compute the expectation value of the energy for each of these temperatures. Plot these as (labeled) vertical lines on the DOS plot. Also mark the infinite temperature energy computed in f. [4pts]

### 1. Energy and Entropy

a. Construct and diagonalize the Hamiltonian for the model above for L=12 (use the helper functions). Plot the normalized distribution of eigenenergies as a function of energy using plt.hist or np.histogram. This serves as a proxy for the density of states (DOS). [5pts]

# Question 2: Diagonal ETH (15 pts)#



We will now verify the (diagonal) ETH for the non-integrable transverse field Ising chain, for a particular local observable $A= \sigma_{L/2}^x$. The statement of diagonal ETH is:

$A_{\alpha \alpha}   = \langle A\rangle_T + O(1/L) +O(e^{-S/2})$ (Equation 1)

where $|\alpha\rangle$ is a many-body eigenstate with eigenenergy $E$, and  $\langle A\rangle_T = \frac{1}{Z}\mbox{Tr } A e^{-H/T}$ is the thermal expectation value of $A$ at a temperature $T$ corresponding to the energy $E$, and $S(E) \propto L$ is the entropy, so that $e^{S(E)}$ is the relevant Hilbert space dimension at energy $E$.

Conditions to verify:
1. The diagonal eigenstate expectation value (EEV) $A_{\alpha\alpha}$ varies smoothly as a function of the energy of the eigenstates.
2. The eigenstate-to-eigenstate fluctuations in the EEVs decrease (exponentially) with increasing system size. Note that this is closely related to point 1. The energy difference between neighboring eigenstates in the middle of the many-body spectrum is exponentially small in $L$. Thus, if $O(E)$ is a smooth function of $E$, we expect differences between nearby eigenstate expectation values to be exponentially close. This is the $O(e^{-S/2})$ part in Equation (1) above. Note that all states within some $O(1)$ energy window $\Delta E$ have the same energy density and hence the same temperature, so $\langle A\rangle_T$ is the same for all states in the window.

2. The eigenstate expectation value of the observable agrees with the expected thermodynamic answer, $\langle A\rangle_T$ at the same temperature. However, note that the difference between the EEV and $\langle A\rangle_T$ only decreases polynomially with $1/L$ (similar in spirit to the agreement between canonical and microcanonical ensembles in standard stat mech, up to poly(1/L) corrections).

Useful references:
1. Srednicki, The approach to thermal equilibrium in quantized chaotic systems; https://arxiv.org/abs/cond-mat/9809360v2

2. Buegeling, Moessner, Haque, Finite-size scaling of eigenstate thermalization; https://arxiv.org/abs/1308.2862

a. Make scatter plots of $A_{\alpha \alpha}$ vs. $E_\alpha$ for $L=8,10,12$. Do you visually see the distributions narrowing with increasing $L$? You may find the hist2d function in matplotlib useful. (5pts)

b. Pick an array of logarithmically spaced temperatures between 0.1 and $10^{10}$ (check out np.logspace). For each temperature in your list, compute $\langle A\rangle_T$ and  $\langle H\rangle_T$. Plot $\langle A\rangle_T$ vs.  $\langle H\rangle_T$ on the scatter plot in (a) for the largest system size. Does the thermal curve track the scatter plot? (5pts)

c. Set $h_z=0$ and repeat a-c. When $h_z=0$, the model is the TFIM which is an integrable non-interacting model after performing a Jordan-Wigner transformation. Thus, we do not expect ETH to hold for this model, and expect fluctuations to be be much larger. Do you see this?  (5pts)

# Question 3: Quenches (25 pts)#

Next, we will consider dynamics starting from general initial states $|\psi_0\rangle$. Thermalization means $\langle \psi(t)|A|\psi(t)\rangle \rightarrow  \langle A\rangle_T$ at late times, where the temperature is set by the energy density of the initial state. The instantaneous fluctuations in time are again supposed to be exponentially small in $L$, scaling as $e^{-S/2}$.  

a. Fix L=10. Pick initial product states $|\psi_0\rangle $ with different expectation values of energies spanning the spectrum. It is easiest to choose product states in the $Z$ basis, which are of the form $|\uparrow \downarrow \cdots \uparrow\rangle$.  Note that the diagonal entries of $H$ are equal to $\langle n|H|n\rangle$ for product states $|n\rangle$ in the $Z$ basis. By sorting the diagonal entries, pick 10 indices $n$ corresponding to (roughly) equally spaced energy expectation values. Indicate these energies as vertical lines on the DOS. (5pts)

b. Obtain $\langle A\rangle_T$ for each of the initial states you picked in a. Note that it is easiest to do this approximately by picking a finely spaced list of temperatures, and obtaining $\langle A\rangle_T$ and  $\langle H\rangle_T$  for this list, as in Question 2. These parametric plot of $\langle A\rangle_T$ vs  $\langle H\rangle_T$ should look smooth. You can then read off the (interpolated) expected thermal $\langle A\rangle_T$s corresponding to the energies of the different initial states in (a). (5pts)

c. Plot $\langle \psi(t)|A|\psi(t)\rangle$ vs. $t$ for each of the initial states in (a) on the same plot using different colors. Add dashed lines indicating the expected thermal values. Do the late-time expectation values agree with the thermal values? (5pts)

d. Fix a particular initial state corresponding to infinite temperature, and plot $\langle \psi(t)|A|\psi(t)\rangle$ vs. $t$ for $L=8,10,12$ on the same plot. Do you see the late time fluctuations narrowing with increasing $L$? Verify that the late-time fluctuations scale as $e^{-S/2}\sim 2^{-L/2}$. (You may find it convenient to start with a $+Y$ product state which is at infinite temperature and can allow you to easily compare across sizes; the gen_state_bloch function will help you obtain such a state). (5pts)

e. No numerics for this part. Expand $\langle \psi(t)|A|\psi(t)\rangle$  in the eigenbasis of the Hamiltonian, separating the sums between time-independent diagonal contributions and the time-dependent off-diagonal ones. Use the ETH ansantz for off-diagonal matrix elements, and assume that your eigenvectors look like random vectors (this is a reasonable approximation for infinite temperature eigenstates which dominate the DOS). By considering the scaling of all the different compnents, show that the instantaneous off-diagonal fluctuation scales as $e^{-S/2}$ for this late time. (5pts)



